In [1]:
import pickle
import h5py
import numpy as np

DATA_DIR = "../../data/logs"

In [2]:
def to_h5():
    data = []
    with open(DATA_DIR + "/data.p", "rb") as f:
        while True:      
            try:  
                data += [pickle.load(f)]
            except EOFError:
                break

    # stack like types together
    nextImages = []
    currentImages = []
    metadata = []
    ids = []

    for record in data:
        nextImages.append(record['nextImage'])
        currentImages.append(record['currentImage'])
        metadata.append([
            record['action'], record['reward'], int(record['died']), record["didBoost"]
        ])
        ids.append(record.get('id', b"00000000-0000-0000-0000-000000000000"))

    nextImages = np.array(nextImages)
    currentImages = np.array(currentImages)
    metadata = np.array(metadata)
    ids = np.array(ids)

    assert nextImages.shape == currentImages.shape
    assert metadata.shape[0] == nextImages.shape[0]

    try:
        # read in existing and append
        with h5py.File(DATA_DIR + '/preprocessed.h5','r') as hf:
            nextImages = np.vstack((hf['nextImage'][...], nextImages))
            currentImages = np.vstack((hf['currentImage'][...], currentImages))
            metadata = np.vstack((hf['metadata'][...], metadata))
            ids = np.vstack((hf['ids'][...], ids))
    except (FileNotFoundError, OSError):
        print("No file found, creating initial h5")
        pass

    # write new
    with h5py.File(DATA_DIR + '/preprocessed.h5','w') as hf:
        hf.create_dataset('currentImage', data=currentImages)
        hf.create_dataset('nextImage', data=nextImages)
        hf.create_dataset('metadata', data=metadata)
        hf.create_dataset("ids", data=ids)

    
to_h5()

No file found, creating initial h5
